In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [1]:
%pwd
%cd C:\Users\Admin\GENAI PROJECTS\2) NLP projects

C:\Users\Admin\GENAI PROJECTS\2) NLP projects


In [3]:
import chardet

# Detect encoding
with open('input_data.csv', 'rb') as f:
    result = chardet.detect(f.read())
    
encoding = result['encoding']
print(f"Detected encoding: {encoding}")

# Read CSV with detected encoding
data = pd.read_csv('input_data.csv', encoding=encoding)

Detected encoding: Windows-1252


In [4]:
##data = pd.read_csv('input_data.csv', encoding='utf-8-sig')

In [5]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [6]:
##pip install transformers

In [7]:
##pip install --upgrade transformers

In [8]:
##pip install torch

In [9]:
from transformers import BertTokenizer##, BertModel
##from transformers.models.bert import BertModel
from transformers import BertModel
import torch
import numpy as np

# Initialize model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

def get_bert_embeddings(texts):
    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :].numpy()
        embeddings.append(cls_embedding)
    return np.vstack(embeddings)

# Compute embeddings for training and testing data
train_embeddings = get_bert_embeddings(train_data['text'])
test_embeddings = get_bert_embeddings(test_data['text'])


C:\Users\Admin\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute similarities between test and training embeddings
similarities = []
for test_embedding in test_embeddings:
    similarity_scores = cosine_similarity([test_embedding], train_embeddings)[0]
    similarities.append(similarity_scores)

# Display similarity scores
for i, test_similarity in enumerate(similarities):
    print(f"Test example {i}:")
    for j, score in enumerate(test_similarity):
        print(f"  Similarity with training example {j}: {score:.4f}")

    # Find and print the most similar training example
    most_similar_idx = np.argmax(test_similarity)
    print(f"  Most similar to training example {most_similar_idx} with a similarity score of {test_similarity[most_similar_idx]:.4f}")
    print()  # For better readability


Test example 0:
  Similarity with training example 0: 0.8423
  Similarity with training example 1: 0.8116
  Similarity with training example 2: 0.8096
  Similarity with training example 3: 0.8354
  Similarity with training example 4: 0.7860
  Similarity with training example 5: 0.7918
  Similarity with training example 6: 0.8457
  Similarity with training example 7: 0.8519
  Similarity with training example 8: 0.7673
  Similarity with training example 9: 0.8294
  Similarity with training example 10: 0.8414
  Similarity with training example 11: 0.7828
  Similarity with training example 12: 0.8335
  Similarity with training example 13: 0.7862
  Similarity with training example 14: 0.7500
  Similarity with training example 15: 0.6980
  Similarity with training example 16: 0.8258
  Similarity with training example 17: 0.7760
  Similarity with training example 18: 0.7230
  Similarity with training example 19: 0.7772
  Similarity with training example 20: 0.7798
  Similarity with training e

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




  Similarity with training example 303: 0.7403
  Similarity with training example 304: 0.8030
  Similarity with training example 305: 0.7545
  Similarity with training example 306: 0.8906
  Similarity with training example 307: 0.7628
  Similarity with training example 308: 0.8939
  Similarity with training example 309: 0.8087
  Similarity with training example 310: 0.7472
  Similarity with training example 311: 0.7892
  Similarity with training example 312: 0.9023
  Similarity with training example 313: 0.7815
  Similarity with training example 314: 0.7956
  Similarity with training example 315: 0.7686
  Similarity with training example 316: 0.7784
  Similarity with training example 317: 0.8606
  Similarity with training example 318: 0.6957
  Similarity with training example 319: 0.8680
  Similarity with training example 320: 0.7791
  Similarity with training example 321: 0.7661
  Similarity with training example 322: 0.8385
  Similarity with training example 323: 0.8603
  Similarity

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

# Encode labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_data['label'])

# Train a classifier
classifier = LogisticRegression()
classifier.fit(train_embeddings, train_labels)


C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [12]:
# Predict labels for test data
test_predictions = classifier.predict(test_embeddings)
test_predictions_labels = label_encoder.inverse_transform(test_predictions)

# Display results
for i, (text, pred_label) in enumerate(zip(test_data['text'], test_predictions_labels)):
    print(f"Test example {i}:")
    print(f"  Text: {text}")
    print(f"  Predicted Label: {pred_label}")
    print()


Test example 0:
  Text: CPK- MB (Creatine Kinase MB)
  Predicted Label: CPK, MB fraction (mass), Serum

Test example 1:
  Text: CD45 IHC, TISSUE/PARAFFIN BLOCK
  Predicted Label: IHC single marker with reporting CD21

Test example 2:
  Text: Thyroxine - Free (FT4)
  Predicted Label: T3, Free Free Tri-Iodothyronine

Test example 3:
  Text: HLA-B27, FLOW CYTOMETRY, BLOOD
  Predicted Label: HLA B27, BLOOD

Test example 4:
  Text: MUCICARMINE STAIN
  Predicted Label: Histochemistry (Special Stain without interpretation) Mucicarmine Stain Tissue / FFPE

Test example 5:
  Text: IHC Marker CD 99
  Predicted Label: IHC single marker with reporting CD68

Test example 6:
  Text: FISH 5Q DELECTION & MONOSOMY/NUMERICAL ABNORMALITY
  Predicted Label: Deletion 5q /Monosomy 5 MDS

Test example 7:
  Text: Cortisol AM
  Predicted Label: Cortisol (Evening)

Test example 8:
  Text: Blood Glucose Random
  Predicted Label: Cortisol, Serum

Test example 9:
  Text: DHEA -So4  (Dehydroepiandrosterone Sulphate

In [13]:
to_be_predicted_data = pd.read_csv('to_be_predicted_data.csv')

In [14]:
to_be_predicted_data

,text,label
0,kidney test,NaN
1,lver test,NaN
2,renal test,NaN
3,kidney_,NaN
4,liver_,NaN
5,complete hemogram,NaN
6,cbc,NaN
7,iron studies,NaN
8,CRP,NaN
9,C reactive protein,NaN


In [15]:
# Function to classify new input text
def classify_text(text, model, tokenizer, classifier, label_encoder):
    embedding = get_bert_embeddings([text])
    prediction = classifier.predict(embedding)
    predicted_label = label_encoder.inverse_transform(prediction)
    return predicted_label[0]


# Example fresh input data
##new_text = "lvr funct"  ---CHANGED NOW DUE TO INPUT FILE BAING TAKEN FROM CSV
index_to_update = 0
for new_text in to_be_predicted_data['text']:
    predicted_label = classify_text(new_text, model, tokenizer, classifier, label_encoder)
    print(f"Predicted label for new text: {predicted_label}")
    to_be_predicted_data.at[index_to_update, 'label'] = predicted_label
    index_to_update+=1


Predicted label for new text: kidney function test
Predicted label for new text: liver function test
Predicted label for new text: kidney function test
Predicted label for new text: kidney function test
Predicted label for new text: liver function test
Predicted label for new text: Complete Hemogram
Predicted label for new text: Gram Stain
Predicted label for new text: Iron Studies
Predicted label for new text: Prolactin
Predicted label for new text: IHC single marker with reporting p53


In [16]:
to_be_predicted_data

,text,label
0,kidney test,kidney function test
1,lver test,liver function test
2,renal test,kidney function test
3,kidney_,kidney function test
4,liver_,liver function test
5,complete hemogram,Complete Hemogram
6,cbc,Gram Stain
7,iron studies,Iron Studies
8,CRP,Prolactin
9,C reactive protein,IHC single marker with reporting p53


In [17]:
to_be_predicted_data.to_csv('to_be_predicted_data.csv')